In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
import os
import numpy as np
import pandas as pd

In [2]:
train = r'C:\Users\HP\Downloads\Pneumonia Detection Dataset\chest_xray\train'
test = r'C:\Users\HP\Downloads\Pneumonia Detection Dataset\chest_xray\test'
val = r'C:\Users\HP\Downloads\Pneumonia Detection Dataset\chest_xray\val'

In [3]:
batch_size=15
image_size=(120,120)

In [4]:
train_datagen= ImageDataGenerator(rotation_range=15,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True,
                                 rescale=1./255) 

test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary")

test_generator = test_datagen.flow_from_directory(
    test,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary")

validation_generator = test_datagen.flow_from_directory(
    val,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary")

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [6]:
model=Sequential()
model.add(Conv2D(30,(3,3),activation='relu',input_shape=(120,120,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(75,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(120,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(220,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 30)      840       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 30)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 30)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 75)        20325     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 75)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 28, 28, 75)        0

In [8]:
model.fit_generator(train_generator,
                   steps_per_epoch=5216//15,
                   epochs=20,
                   verbose=1,
                   validation_data=test_generator,
                   validation_steps=624//15)

C:\Users\HP\AppData\Local\Temp/ipykernel_9744/2356288912.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/20
347/347 [==============================] - 282s 805ms/step - loss: 0.5036 - accuracy: 0.7583 - val_loss: 0.7081 - val_accuracy: 0.6894
Epoch 2/20
347/347 [==============================] - 218s 629ms/step - loss: 0.3581 - accuracy: 0.8456 - val_loss: 0.5094 - val_accuracy: 0.7984
Epoch 3/20
347/347 [==============================] - 212s 610ms/step - loss: 0.3115 - accuracy: 0.8762 - val_loss: 0.6919 - val_accuracy: 0.7691
Epoch 4/20
347/347 [==============================] - 214s 616ms/step - loss: 0.2662 - accuracy: 0.8941 - val_loss: 0.4126 - val_accuracy: 0.8049
Epoch 5/20
347/347 [==============================] - 214s 617ms/step - loss: 0.2629 - accuracy: 0.8993 - val_loss: 0.4296 - val_accuracy: 0.8049
Epoch 6/20
347/347 [==============================] - 219s 630ms/step - loss: 0.2268 - accuracy: 0.9142 - val_loss: 0.3343 - val_accuracy: 0.8585
Epoch 7/20
347/347 [==============================] - 214s 616ms/step - loss: 0.2206 - accuracy: 0.9177 - val_loss: 0.7086 -

In [14]:
model.save('pneumonia.h5')

In [1]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [2]:
model = load_model('pneumonia.h5')

In [11]:
from skimage.transform import resize
import cv2
def detect(frame):
    try:
        img = resize(frame,(120,120))
        img = np.expand_dims(img,axis=0)
        prediction = model.predict(img)
        print(prediction)
        prediction = (model.predict(img) > 0.5).astype("int32")
        print(prediction)
    except AttributeError:
        print("Shape not found")
frame = cv2.imread(r"C:\Users\HP\Desktop/test4.jpeg")
data = detect(frame)

[[0.9999974]]
[[1]]
